# Dependencies

In [ ]:
# Due to reasons unknown to the author, Google Colab's default pip version 19.3.1 does not work well for SentencePiece in this case.
# "!" is necessary to run non-python command in Google Colab.
!python -m pip install 'pip==20.3.3'
!pip install --upgrade PyYAML
# The tokenizer of MarianMT model from Hugging Face is based on SentencePiece.
!pip install transformers sentencepiece datasets

     |████████████████████████████████| 1.5MB 7.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 636 kB 8.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 237 kB 8.3 MB/s 
     |████████████████████████████████| 1.2 MB 14.9 MB/s 
     |████████████████████████████████| 2.5 MB 43.7 MB/s 
     |████████████████████████████████| 3.3 MB 34.9 MB/s 
     |████████████████████████████████| 114 kB 52.9 MB/s 
     |████████████████████████████████| 895 kB 49.5 MB/s 
     |████████████████████████████████| 243 kB 51.3 MB/s 


In [ ]:
import transformers
import sentencepiece
model_checkpoint = "Helsinki-NLP/opus-tatoeba-es-zh"

## What do we need

In [ ]:
'''
Credit to: https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()
'''

'\nCredit to: https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb\ntrainer = Seq2SeqTrainer(\n    model,\n    args,\n    train_dataset=tokenized_datasets["train"],\n    eval_dataset=tokenized_datasets["validation"],\n    data_collator=data_collator,\n    tokenizer=tokenizer,\n    compute_metrics=compute_metrics\n)\ntrainer.train()\n'

***class* Seq2SeqTrainer**

https://huggingface.co/transformers/main_classes/trainer.html#id1
https://huggingface.co/transformers/main_classes/trainer.html#seq2seqtrainer

**model**: torch.nn.modules.module.Module

**tokenizer**: transformers.MarianTokenizer

**tokenized dataset**: train & evaluation

*   train_dataset (tokenized): torch.utils.data.dataset.Dataset
*   eval_dataset (tokenized): torch.utils.data.dataset.Dataset

**training configuration**: *class* Seq2SeqTrainingArguments

**data collator**: to be able to build batches, in our case, we need *class* transformers.data.data_collator.DataCollatorForSeq2Seq




## Model information

In [ ]:
from transformers import MarianConfig, MarianTokenizer, MarianMTModel
configurator = MarianConfig.from_pretrained(model_checkpoint)
tokenizer = MarianTokenizer.from_pretrained(model_checkpoint,use_fast=False)
'''
MarianMTModel: https://huggingface.co/transformers/model_doc/marian.html#marianmodel
This model inherits from PreTrainedModel.
Parameter: config (MarianConfig) – Model configuration class with all the parameters of the model.
  Initializing with a config file does not load the weights associated with the model, only the configuration.
  Check out the from_pretrained() method to load the model weights.
'''
model = MarianMTModel.from_pretrained(model_checkpoint)

In [ ]:
# we can use an instance of the configurator class to get details of our model.
configurator

MarianConfig {
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      65000
    ]
  ],
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 65000,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 0,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
 

In [ ]:
# transformers 4.5.0 no longer supports swish as selectable activation function.
# configurator.activation_function = 'gelu'/'relu'/'silu'/'gelu_new'

In [ ]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
   

In [ ]:
tokenizer
# vocab_size in the tokenizer is the same as the input number for embedding layer and the final softmax output number
# (shared): Embedding(65001, 512, padding_idx=65000)
# and (lm_head): Linear(in_features=512, out_features=65001, bias=False)

PreTrainedTokenizer(name_or_path='Helsinki-NLP/opus-tatoeba-es-zh', vocab_size=65001, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'})

We do not change the vocabulary size here.

#Creating Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset
raw_data = load_dataset('csv',data_files='/content/drive/MyDrive/All.csv')

Using custom data configuration default-d484fbe96b45b0ba


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d484fbe96b45b0ba/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [ ]:
raw_data
#A datasets.DatasetDict object with only the train split at the moment
#Note how the first row of the CSV file becomes the features. They are essentially the columns' name

DatasetDict({
    train: Dataset({
        features: ['es', 'zh'],
        num_rows: 9972
    })
})

In [ ]:
train_split = raw_data['train']

In [ ]:
train_split
#We extracted a datasets.Dataset object item from the dataset.DatasetDict instance. 

Dataset({
    features: ['es', 'zh'],
    num_rows: 9972
})

In [ ]:
# First shuffle the dataset then
# split the dataset
# and return a datasets.DatasetDicct object with two random train and test subsets
# where the test split contains 1000 rows.
DS = train_split.train_test_split(test_size=1000,shuffle=True)

In [ ]:
DS

DatasetDict({
    train: Dataset({
        features: ['es', 'zh'],
        num_rows: 8972
    })
    test: Dataset({
        features: ['es', 'zh'],
        num_rows: 1000
    })
})

### Extract test split for later use

In [ ]:
test = DS['test']

In [ ]:
zh = open('Test_zh.txt','w')
es = open('Test_es.txt','w')

In [ ]:
for row in test:
  src = row['es']
  tgt = row['zh']
  es.write(src+'\n')
  zh.write(tgt+'\n')
es.close()
zh.close()

### Visualizing dataset examples:

In [ ]:
# credit to https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
# Show randomly 5 rows from the test split.
show_random_elements(DS['test'],5)

,es,zh
0,"La enajenación que de los bienes inmuebles sujetos a reserva hubiere hecho el viudo o la viuda después de contraer segundo matrimonio subsistirá únicamente si a su muerte no quedan hijos ni descendientes del primero, sin perjuicio de lo dispuesto en la Ley Hipotecaria.",鳏寡在缔结第二次婚姻之后让与不动产保留义务的，仅在其死亡时前次婚姻无子女和卑亲属时有效，且不得有悖抵押法的规定。
1,"El que se obliga a hacer una obra por piezas o por medida, puede exigir del dueño que la reciba por partes y que la pague en proporción.",如工程包括数个建筑物或按度量计算时，定作人可选择得逐步验收并逐步交费。
2,"Si la extradición es solicitada concurrentemente por varios Estados, bien por el mismo delito o por delitos diferentes, la Parte requerida tomará una decisión sobre las solicitudes recibidas, notificándolo a la Parte requirente.",如果多个国家就相同或者不同犯罪针对同一 人提出引渡请求，被请求方应当自主对这些请求作出决定，并告知请求方。
3,"Si la deuda produce interés, no podrá estimarse hecho el pago por cuenta del capital mientras no estén cubiertos los intereses.",债务产生的利息，在偿还本金但未支付利息时不视为债务已经清偿。
4,Esta regla no se aplicará a los contratos relativos a inmuebles situados en el extranjero.,但合同标的为在国外的不动产的，不适用本规定。


# Dataset Preparation: Tokenization

In [ ]:
# credit to https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb
def preprocess_data(dataset):
  src = [row for row in dataset['es']]
  tgt = [row for row in dataset['zh']]
  model_inputs = tokenizer(src, truncation=True)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_data = DS.map(preprocess_data,batched=True)
# .map() method applies the functions to all splits of the DatasetDict object.

In [ ]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'es', 'input_ids', 'labels', 'zh'],
        num_rows: 8972
    })
    test: Dataset({
        features: ['attention_mask', 'es', 'input_ids', 'labels', 'zh'],
        num_rows: 1000
    })
})

In [ ]:
tokenized_data['train'][50]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'es': 'Serán de cargo de la sociedad de gananciales los gastos que se originen por alguna de las siguientes causas:',
 'input_ids': [110,
  6886,
  2,
  1173,
  2,
  4,
  574,
  2,
  45720,
  1955,
  13,
  472,
  11,
  20,
  51,
  17713,
  5481,
  65,
  25,
  2094,
  2,
  15,
  673,
  2262,
  39,
  0],
 'labels': [8, 965, 2435, 971, 1214, 4851, 39, 0],
 'zh': '共同财产用于以下用途：'}

In [ ]:
# We can use tokenizer to chekc which token each id represents.
tokenizer.convert_ids_to_tokens(2770)

'▁deberán'

In [ ]:
tokenizer.convert_ids_to_tokens(2918)

'至少'

# Training Configuration

In [ ]:
import os
os.chdir('/content/drive/MyDrive')

In [ ]:
!nvidia-smi

Fri Jun 18 16:15:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import Seq2SeqTrainingArguments
batch_size = 8
args = Seq2SeqTrainingArguments(
    output_dir='huggingface_outputs',
    logging_dir='huggingface_logging',
    evaluation_strategy='steps',
    prediction_loss_only = True,
    logging_steps=400,
    learning_rate=2e-5,
    warmup_steps=2000,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=10,
    num_train_epochs=10,
    load_best_model_at_end=True,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
earlystopping = transformers.EarlyStoppingCallback(early_stopping_patience=8)

# Train

In [ ]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

Data collators are objects that will form a batch by using a list of dataset elements as input.

To be able to build batches, data collators may apply some processing (like padding).

DataCollatorForSeq2Seq will dynamically pad the inputs received, as well as the labels.

References: https://huggingface.co/transformers/main_classes/data_collator.html

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[earlystopping]
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
400,2.958400,2.679962
800,2.640200,2.501655
1200,2.503800,2.390669


Step,Training Loss,Validation Loss
400,2.958400,2.679962
800,2.640200,2.501655
1200,2.503800,2.390669
1600,2.327900,2.299865
2000,2.225800,2.234325
2400,2.106100,2.196070
2800,1.927900,2.156900
3200,1.905900,2.124531
3600,1.749100,2.122658
4000,1.630100,2.116931


TrainOutput(global_step=8800, training_loss=1.725204648104581, metrics={'train_runtime': 1255.342, 'train_samples_per_second': 71.471, 'train_steps_per_second': 8.938, 'total_flos': 3928473063137280.0, 'train_loss': 1.725204648104581, 'epoch': 7.84})

The training automatically stopped because the best metric (validation loss in this case) we got at step 5600 has not been better after 8 evaluations.

In [ ]:
trainer.save_model(output_dir='huggingface_model')

# Upload the Fine-tuned Model to Hugging Face

In [ ]:
!transformers-cli login
# after executing this line, one needs to type in his/her username and password of Hugging Face account.

The method trainer.push_to_hub() requires Git Lfs installed.

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 47 not upgraded.
Need to get 6,229 kB of archives.
After this operation, 14.5 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 2.13.3 [6,229 kB]
Fetched 6,229 kB in 0s (16.2 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/per

In [ ]:
!git config --global user.email "guocheng.zhu@e-campus.uab.cat"
!git config --global user.name "Guocheng"

In [ ]:
trainer.push_to_hub(repo_name='HelsinkiNLP-FineTuned-Legal-es-zh')

'https://huggingface.co/guocheng98/HelsinkiNLP-FineTuned-Legal-es-zh/commit/967e848db26226963d39694605eb68c8c6f59a31'